In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!pip install pytorch-lightning

     |████████████████████████████████| 819kB 5.7MB/s 
     |████████████████████████████████| 112kB 21.9MB/s 
     |████████████████████████████████| 276kB 17.8MB/s 
     |████████████████████████████████| 829kB 18.0MB/s 
     |████████████████████████████████| 1.3MB 27.8MB/s 
     |████████████████████████████████| 296kB 36.2MB/s 
     |████████████████████████████████| 143kB 22.8MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=20e12e25e5be1316bd39f05a5c0e4f645ec07ad92777a394b6b02c2f60d582eb
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=341a125d2999d6a14c0b5b47642cbbffc6d7306ace5fe2aa1e523037af400e4e
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAML future
  Found existing installation: PyYAML 3.13
    U

In [ ]:
import os
import torch 
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch import Tensor
from torch.nn import BCEWithLogitsLoss
from pathlib import Path
from PIL import Image
import pytorch_lightning as pl
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.metrics import Accuracy
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint
from typing import Callable, Dict, List, Tuple, Union
import torch.nn.functional as F
import random



In [ ]:
BASE = Path("/content/gdrive/MyDrive/covid")
ROOT = BASE / "images"

COVID_TEXT_LABELS = Path("/content/gdrive/MyDrive/covid/covid_text_labels")
COVID_TRAIN = COVID_TEXT_LABELS / "covid_train_labels.txt"
COVID_VAL = COVID_TEXT_LABELS / "covid_val_labels.txt"
COVID_TEST = COVID_TEXT_LABELS / "covid_test_labels.txt"

NONCOVID_TEXT_LABELS = Path("/content/gdrive/MyDrive/covid/noncovid_text_labels")
NONCOVID_TRAIN = NONCOVID_TEXT_LABELS / "noncovid_train_labels.txt"
NONCOVID_VAL = NONCOVID_TEXT_LABELS / "noncovid_val_labels.txt"
NONCOVID_TEST = NONCOVID_TEXT_LABELS / "noncovid_test_labels.txt"

BATCH_SIZE = 8
EPOCHS = 3


INPUT_SIZE = (224, 224)


In [ ]:
def read_text_labels(text_path: str) -> List[str]:
    """read file in `txt_path`, stripping away leading and trailing whitespace"""
    with open(text_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data


In [ ]:
class CovidDataset(Dataset):
    def __init__(self, root_dir, text_COVID, text_NonCOVID, transform):
        self.root_dir = root_dir
        self.text_path = [text_COVID, text_NonCOVID]
        self.classes = ["CT_COVID", "CT_NonCOVID"]
        self.covid_images = read_text_labels(text_COVID)
        self.noncovid_images = read_text_labels(text_NonCOVID)
        self.covid_path = Path(root_dir).resolve() / "CT_COVID"
        self.noncovid_path = Path(root_dir).resolve() / "CT_NonCOVID"
        self.covid_images = [self.covid_path / img for img in self.covid_images]
        self.noncovid_images = [self.noncovid_path / img for img in self.noncovid_images]
        self.images = self.covid_images + self.noncovid_images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):

        if torch.is_tensor(index):
            index = index.tolist()     

        img_path = self.images[index]  
        parent = img_path.parent.name
        label = 1 if parent == "CT_COVID" else 0
        
        image = Image.open(img_path).convert("RGB")


        if self.transform:
            image = self.transform(image)
            
        return image, label

In [ ]:
def covid_dataset_helper():
    def get_transforms(input_size: int = INPUT_SIZE):

        # The mean and std values is suggested by torchvision.model documentation
        #link: https://pytorch.org/vision/0.8/models.html
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      
        train_transform = transforms.Compose(
            [
                # transforms.RandomResizedCrop(input_size),
                # transforms.RandomHorizontalFlip(),
                # transforms.ToTensor(),
                # normalize,
                transforms.Resize(256),
                transforms.RandomResizedCrop((224), scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                #     transforms.RandomRotation(90),
                # random brightness and random contrast
                transforms.ColorJitter(brightness=0.2, contrast=0.2),
                transforms.ToTensor(),
                normalize,
            ]
        )

        val_transform = transforms.Compose(
            [
                # transforms.Resize(input_size),
                # transforms.CenterCrop(input_size),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                normalize,
            ]
        )
        return train_transform, val_transform

    train_transform, val_transform = get_transforms()
    trainset = CovidDataset(
        root_dir=str(ROOT),
        text_COVID=str(COVID_TRAIN),
        text_NonCOVID=str(NONCOVID_TRAIN),
        transform=train_transform,
    )
    valset = CovidDataset(
        root_dir=str(ROOT),
        text_COVID=str(COVID_VAL),
        text_NonCOVID=str(NONCOVID_VAL),
        transform=val_transform,
    )
    testset = CovidDataset(
        root_dir=str(ROOT),
        text_COVID=str(COVID_TEST),
        text_NonCOVID=str(NONCOVID_TEST),
        transform=val_transform,
    )

    return trainset, valset, testset

    # def train_val_test_dataloader():
    #     train_transform, val_transform = get_transforms()
    #     trainset = CovidDataset(
    #         root_dir=str(ROOT),
    #         text_COVID=str(COVID_TRAIN),
    #         text_NonCOVID=str(NONCOVID_TRAIN),
    #         transform=train_transform,
    #     )
    #     valset = CovidDataset(
    #         root_dir=str(ROOT),
    #         text_COVID=str(COVID_VAL),
    #         text_NonCOVID=str(NONCOVID_VAL),
    #         transform=val_transform,
    #     )
    #     testset = CovidDataset(
    #         root_dir=str(ROOT),
    #         text_COVID=str(COVID_TEST),
    #         text_NonCOVID=str(NONCOVID_TEST),
    #         transform=val_transform,
    #     )

    #     train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)
    #     val_loader = DataLoader(valset, batch_size=BATCH_SIZE, drop_last=True, shuffle=False)
    #     test_loader = DataLoader(testset, batch_size=BATCH_SIZE, drop_last=True, shuffle=False)

            
    #     return train_loader, val_loader, test_loader
    # return train_val_test_dataloader()


In [ ]:
class ResnetTransferLearning(LightningModule):
    def __init__(self, input_shape, learning_rate=1e-3):
        super().__init__()
        
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.input_shape = input_shape
        self.accuracy = pl.metrics.Accuracy()
        


        self.feature_extractor = models.resnet18(pretrained=True)
        self.feature_extractor.eval()
        """ we are using the convolutional output of each model, 1000 is the convolutional output of resnet18 """
        self.classfier = nn.Linear(1000, 1)

    def forward(self, x):
        self.feature_extractor.eval()
        x = self.feature_extractor(x)
        x = x.unsqueeze(1)
        x = self.classfier(x)
        x = x.squeeze()
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        output = self(x)
        criterion = BCEWithLogitsLoss()
        try:
            loss = criterion(output.unsqueeze(1), y.unsqueeze(1).float())
        except BaseException as e:
            
            raise e
        
        logits = torch.sigmoid(output)
        pred = torch.round(logits)
        acc = torch.mean((pred==y).float())
        self.log('train_acc_step', acc, prog_bar=True)
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):         
        x, y = batch
        output = self(x)
        criterion = BCEWithLogitsLoss()
        logits = torch.sigmoid(output)
        pred = torch.round(logits)
        acc = torch.mean((pred==y).float())
        self.log('val_acc', acc, prog_bar=True)
        loss = criterion(output.unsqueeze(1), y.unsqueeze(1).float())
        self.log('val_loss', loss)
        # self.valid_acc(logits, y)
        # self.log('valid_acc', self.valid_acc, on_step=True, on_epoch=True)
    
        return loss

    def test_step(self,batch,batch_idx):
        x, y = batch
        output = self(x)
        criterion = BCEWithLogitsLoss()
        logits = torch.sigmoid(output)
        pred = torch.round(logits)
        self.test_pred.append(pred)
        acc = torch.mean((pred==y).float())
        self.log('test_acc', acc, prog_bar=True)
        loss = criterion(output.unsqueeze(1), y.unsqueeze(1).float())
        self.log('test_loss', loss)
    
        

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)
            

In [ ]:
if __name__ == "__main__":
    # train_loader, val_loader, test_loader = covid_dataset_helper()
    trainset, valset, testset = covid_dataset_helper()
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)
    val_loader = DataLoader(valset, batch_size=BATCH_SIZE, drop_last=True, shuffle=False)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, drop_last=True, shuffle=False)

 
    

In [ ]:
def bagging_loader(dataset: CovidDataset, percent: float = 0.6, shuffle: bool = False) -> DataLoader:
    size = int(round(percent * len(dataset)))
    
    # idx = torch.randperm(size)
    idx = random.sample(range(0, len(dataset)), size)
    
    
    return DataLoader(dataset,sampler=SubsetRandomSampler(idx), batch_size=BATCH_SIZE, drop_last=True, shuffle=shuffle)

[3, 0, 2]

In [ ]:
bag_train_loader = bagging_loader(trainset)
bag_val_loader = bagging_loader(valset)

In [ ]:
bag_train_loader_1 = bagging_loader(trainset)
bag_val_loader_1 = bagging_loader(valset)
bag_test_loader_1 = bagging_loader(testset)

bag_train_loader_2 = bagging_loader(trainset)
bag_val_loader_2 = bagging_loader(valset)
bag_test_loader_2 = bagging_loader(testset)

bag_train_loader_3 = bagging_loader(trainset)
bag_val_loader_3 = bagging_loader(valset)
bag_test_loader_3 = bagging_loader(testset)

bag_train_loader_4 = bagging_loader(trainset)
bag_val_loader_4 = bagging_loader(valset)
bag_test_loader_4 = bagging_loader(testset)


bag_train_loader_5 = bagging_loader(trainset)
bag_val_loader_5 = bagging_loader(valset)
bag_test_loader_5 = bagging_loader(testset)


In [ ]:
# checkpoint_callback = ModelCheckpoint(dirpath='/content/gdrive/MyDrive/covid/checkpoint')

# model = ResnetTransferLearning((3,224,224), 2)
# trainer = pl.Trainer(callbacks=checkpoint_callback, max_epochs=3,progress_bar_refresh_rate=25,gpus=1)
# trainer.fit(model, bag_train_loader, bag_val_loader)

# best_model = ResnetTransferLearning.load_from_checkpoint(checkpoint_callback.best_model_path)


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


GPU available: True, used: True
TPU available: None, using: 0 TPU cores



  | Name              | Type     | Params
-----------------------------------------------
0 | accuracy          | Accuracy | 0     
1 | feature_extractor | ResNet   | 11.7 M
2 | classfier         | Linear   | 1.0 K 
-----------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.762    Total estimated model params size (MB)


In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath='/content/gdrive/MyDrive/covid/checkpoint')

model_1 = ResnetTransferLearning((3,224,224), 2)
trainer_1 = pl.Trainer(callbacks=checkpoint_callback,max_epochs=3,progress_bar_refresh_rate=25,gpus=1)
trainer_1.fit(model_1, bag_train_loader_1, bag_val_loader_1)

best_model = ResnetTransferLearning.load_from_checkpoint(checkpoint_callback.best_model_path)



GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name              | Type     | Params
-----------------------------------------------
0 | accuracy          | Accuracy | 0     
1 | feature_extractor | ResNet   | 11.7 M
2 | classfier         | Linear   | 1.0 K 
-----------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.762    Total estimated model params size (MB)


In [ ]:
best_model.freeze()

In [ ]:
predictions = best_model()

TypeError: ignored